# Part 2: Creating Database and Computing Spatial Statistics

This is part 2 of our 3-part series on Retrieving, Analyzing and Visualizing georeferenced data of earthquakes. We are using the CSV file of the [selected dataset](https://raw.github.com/vincentarelbundock/Rdatasets/master/doc/datasets/quakes.html). This dataset contains location data (longitude and latitude), depth and magnitude of earthquake events that occurred near [Fiji](https://es.wikipedia.org/wiki/Fiyi) since 1964. Next, we will create a database and populate it using this CSV file. Finally, we will query the database and compute some statistics.

We will import all the required Python libraries.

In [2]:
import sqlite3
import requests
import csv
import ssl
from contextlib import closing
import numpy as np
import itertools

## Exploring the dataset

Although we can use [pandas](https://pandas.pydata.org/) to explore our dataset, the challenge of this exercise is to use standard Python libraries. First,  we will locate the URL of the CSV file containing the data, next we will examine the dataset content.

We will use the HTML link of the selected dataset to obtain the URL of its CSV file

In [2]:
with open('datasets.csv', 'rt') as f:
     reader = csv.reader(f, delimiter=',') 
     for row in reader:
          for field in row:
              if field == "https://raw.github.com/vincentarelbundock/Rdatasets/master/doc/datasets/quakes.html":
                  url = row[10]
print('The URL of the CSV file is: ', url)

The URL of the CSV file is:  https://raw.github.com/vincentarelbundock/Rdatasets/master/csv/datasets/quakes.csv


Let's review the content of the dataset. 

According to the `quakes.html` document, the description of the column names is as follows:

* ***lat***: Latitude of event
* ***long***: Longitude
* ***depth***: Depth (km)
* ***stations***: Number of stations reporting

In [57]:
csv_url = url

with requests.Session() as s:
    download = s.get(csv_url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',', quotechar='"')
    datalist = list(cr)
    for row in datalist:
        print(row)

['', 'lat', 'long', 'depth', 'mag', 'stations']
['1', '-20.42', '181.62', '562', '4.8', '41']
['2', '-20.62', '181.03', '650', '4.2', '15']
['3', '-26', '184.1', '42', '5.4', '43']
['4', '-17.97', '181.66', '626', '4.1', '19']
['5', '-20.42', '181.96', '649', '4', '11']
['6', '-19.68', '184.31', '195', '4', '12']
['7', '-11.7', '166.1', '82', '4.8', '43']
['8', '-28.11', '181.93', '194', '4.4', '15']
['9', '-28.74', '181.74', '211', '4.7', '35']
['10', '-17.47', '179.59', '622', '4.3', '19']
['11', '-21.44', '180.69', '583', '4.4', '13']
['12', '-12.26', '167', '249', '4.6', '16']
['13', '-18.54', '182.11', '554', '4.4', '19']
['14', '-21', '181.66', '600', '4.4', '10']
['15', '-20.7', '169.92', '139', '6.1', '94']
['16', '-15.94', '184.95', '306', '4.3', '11']
['17', '-13.64', '165.96', '50', '6', '83']
['18', '-17.83', '181.5', '590', '4.5', '21']
['19', '-23.5', '179.78', '570', '4.4', '13']
['20', '-22.63', '180.31', '598', '4.4', '18']
['21', '-20.84', '181.16', '576', '4.5', '17'

## Creating the database and table

We choose to create a [sqlite](https://www.sqlite.org/index.html) database because is lightweight in terms of setup, database administration and required resource. Once the database is created, we will create a table named `quakes` with the same column names of the dataset examined previously. To do that we will use the [SQL CREATE TABLE Statement](https://www.w3schools.com/sql/sql_create_table.asp).

In [4]:
# create database
conn = sqlite3.connect('eartquakes.sqlite')
cur = conn.cursor()

In [5]:
# create table
cur.execute('''CREATE TABLE IF NOT EXISTS quakes
    (id INTEGER PRIMARY KEY, lat REAL, lon REAL, depth INTEGER,
    mag REAL, stations INTEGER)''')

## Populating the database with earthquake data

In this part, we will get and read the dataset for the URL and use these data to populate the database. Then we will drop some line codes and SQL statements to calculate: total earthquakes, average magnitude and depth, and strong and first minor earthquake.

In [6]:
%%capture

url = url
# get and read data (on line)
with closing(requests.get(url, stream=True)) as r:
    f = (line.decode('utf-8') for line in r.iter_lines())
    reader = csv.reader(f, delimiter=',', quotechar='"')
    next(reader)
    
    # populate the database
    list = []
    for row in reader:
        list.append(row)
    for item in list:
        cur.execute('INSERT OR IGNORE INTO quakes (id, lat, lon, depth, mag, stations) VALUES ( ?, ?, ?, ?, ?, ?)', ( item[0],item[1],item[2],item[3],item[4],item[5], ) )
    conn.commit()
    
    # total earthquakes
    cur.execute("SELECT COUNT(*) from quakes")
    for row in cur:
        count = row[0]
    
    # average magnitude
    cur.execute("SELECT AVG(mag) from quakes")
    for row in cur:
        magAvg = row[0]
    
    # average depth
    cur.execute("SELECT AVG(depth) from quakes")
    for row in cur:
        depAvg = row[0]
    
    # strong and first minor earthquake in the dataset
    cur.execute("SELECT MAX(mag), id, lat, lon, depth FROM quakes")
    for row in cur:
        mag = row[0]
        id = row[1]
        lat = row[2]
        lon = row[3]
        depth = row[4]
    cur.execute("SELECT MIN(mag), id, lat, lon, depth FROM quakes")
    for row in cur:
        mag2 = row[0]
        id2 = row[1]
        lat2 = row[2]
        lon2 = row[3]
        depth2 = row[4]

    # id and magnitude dictionary
    cur.execute("SELECT id, mag FROM quakes")
    dict = {rows[0]:rows[1] for rows in cur}
    cur.close()

### Printing the results

In [16]:
print()
print("Number of earthquakes: "+str(count)+"\n")
print("Average magnitude (Richter): "+str(round(magAvg,1))+"\n")
print("Average depth (km): "+str(round(depAvg,2))+"\n")
print("Strong earthquake: ID:"+str(id)+", latitude:"+str(lat)+", longitude:"+str(lon)+\
      ", magnitude(Richter):"+str(mag)+", depth(km):"+str(depth)+"\n")
print("First Minor earthquake: ID:"+str(id2)+", latitude:"\
      +str(lat2)+", longitude:"+str(lon2)+", magnitude(Richter):"+str(mag2)+\
      ", depth (km):"+str(depth2)+"\n")


Number of earthquakes: 1000

Average magnitude (Richter): 4.6

Average depth (km): 311.37

Strong earthquake: ID:152, latitude:-15.56, longitude:167.62, magnitude(Richter):6.4, depth(km):127

First Minor earthquake: ID:5, latitude:-20.42, longitude:181.96, magnitude(Richter):4.0, depth (km):649



## Calculating Distances

Next, we will calculate the distance between the largest and smallest earthquake magnitude. To do that we will use the [Haversine Distance](https://en.wikipedia.org/wiki/Haversine_formula) which calculates the shortest distance between two coordinates. This is pretty similar to the [Euclidian Distance](https://en.wikipedia.org/wiki/Euclidean_distance), except it allows us to account for the spherical shape of the Earth.

In [15]:
# Haversine distance formula
import math
x_dist = math.radians(lon2-lon)
y_dist = math.radians(lat2-lat)
y1_rad = math.radians(lat)
y2_rad = math.radians(lat2)
a = math.sin(y_dist/2)**2 + math.sin(x_dist/2)**2 \
* math.cos(y1_rad) * math.cos(y2_rad)
c = 2*math.asin(math.sqrt(a))
distance = c * 6371 # kilometers
print()
print("Distance between the strongest earthquake (ID:"+str(id) +") and weakest earthquake \
(ID:"+str(id2)+"): "+str(round(distance,2))+" km.\n")


Distance between the strongest earthquake (ID:152) and weakest earthquake (ID:5): 1609.06 km.



## Classifying Earthquakes by Magnitude

In this section, we are going to classify the earthquakes by magnitude according to the [Mercalli Intensity Scale](https://en.wikipedia.org/wiki/Modified_Mercalli_intensity_scale). We will iterate through the dictionary created previously containing id and magnitude data of each event, then we will filter these events according to the Mercalli Scale threshold and calculate the percentage of occurrence.

In [236]:
# dictionary length
dictLen= len(dict)

# calculate percentage mercaly intensity I
mer_I = [item for item, occurrences in dict.items() if occurrences < 2.0]
merIPerc = round((len(mer_I)/dictLen)*100, 2)

# calculate percentage mercaly intensity II
mer_II = [item for item, occurrences in dict.items() if occurrences >= 2.0 and occurrences <= 2.9 ]
merIIPerc = round((len(mer_II)/dictLen)*100,2)

# calculate percentage mercaly intensity III
mer_III = [item for item, occurrences in dict.items() if occurrences >= 3.0 and occurrences <= 3.9 ]
merIIIPerc = round((len(mer_III)/dictLen)*100,2)

# calculate percentage mercaly intensity IV
mer_IV = [item for item, occurrences in dict.items() if occurrences >= 4.0 and occurrences <= 4.9 ]
merIVPerc = round((len(mer_IV)/dictLen)*100,2)

# calculate percentage mercaly intensity V
mer_V = [item for item, occurrences in dict.items() if occurrences >= 5.0 and occurrences <= 5.9 ]
merVPerc = round((len(mer_V)/dictLen)*100,2)

# calculate percentage mercaly intensity VI
mer_VI = [item for item, occurrences in dict.items() if occurrences >= 6.0 and occurrences <= 6.9 ]
merVIPerc = round((len(mer_VI)/dictLen)*100,2)

# calculate percentage mercaly intensity VII
mer_VII = [item for item, occurrences in dict.items() if occurrences >= 7.0 and occurrences <= 7.9 ]
merVIIPerc = round((len(mer_VII)/dictLen)*100,2)

# calculate percentage mercaly intensity VIII
mer_VIII = [item for item, occurrences in dict.items() if occurrences >= 8.0]
merVIIIPerc = round((len(mer_VIII)/dictLen)*100,2)

### Printing the results

We can observe that ***80 percent*** of the earthquakes in the study area are of [Mecalli Intensity Scale](https://en.wikipedia.org/wiki/Modified_Mercalli_intensity_scale) ***IV-V***. These kind of events are classified as light to moderate earthquakes. According to [Wikipedia](https://en.wikipedia.org/wiki/Modified_Mercalli_intensity_scale) they are felt indoors by many (IV. Light) and felt by nearly everyone (V. Moderate).

In [235]:
# list to store percentages
perlst = [merIPerc, merIIPerc, merIIIPerc, merIVPerc, merVPerc, merVIPerc, merVIIPerc, merVIIIPerc]

# list of scale labels
lablst = ['I', 'II–III', 'III–IV', 'IV-V', 'V-VI', 'VI-VII', 'VII-VIII', 'VIII or higher']

    
for perc, label in zip(perlst, lablst):
    print('{} % of earthquakes are Mercalli Intensity Scale {}\n'.format(perc, label))

0.0 % of earthquakes are Mercalli Intensity Scale I

0.0 % of earthquakes are Mercalli Intensity Scale II–III

0.0 % of earthquakes are Mercalli Intensity Scale III–IV

80.2 % of earthquakes are Mercalli Intensity Scale IV-V

19.3 % of earthquakes are Mercalli Intensity Scale V-VI

0.5 % of earthquakes are Mercalli Intensity Scale VI-VII

0.0 % of earthquakes are Mercalli Intensity Scale VII-VIII

0.0 % of earthquakes are Mercalli Intensity Scale VIII or higher



***Execute `map_earthquakes.ipynb` to built a web map depicting earthquakes events.***